# Skin Tone Check
### This notebook implements a simple way to check whether the clothing transfer network generates skin with color that matches the input image.  If not, the image can either not be returned to the user or an image from a backup method (e.g. a 2D overlay) can be returned.

### Usage: To replicate the results of this notebook, run the cells after setting up the repository and running the test example as defined in the README. Optionally run the human parsing step on the output image as well, if using the `out_seg_path` flag in `get_skintone_similarity`, as described below

In [1]:
import numpy as np
from PIL import Image
from scipy import io
from scipy import misc

### Function that returns a mask that selects just the arms of the image

In [2]:
def get_arm_mask(image_path):
    # Load segmentations and keep only the arms
    mask = io.loadmat(image_path)
    mask = misc.imresize(mask['segment'], (256,192))
    mask = np.array(((mask==14)|(mask==15)), dtype='uint8')
    mask *= 255 # Follow PIL mask conventions (0=mask, 255=keep)
    return mask

### Returns a unit-normalized color histogram of the masked image.
#### The histogram corresponds to a 768-dimensional (256 pixel values * 3 channels) vector representation of the color

In [3]:
def get_color_histogram(image, mask):
    im = Image.fromarray(image)
    mask = Image.fromarray(mask)
    hist = im.histogram(mask)
    # Normalize histogram
    hist = np.asarray(hist, dtype=float)
    hist /= np.linalg.norm(hist)
    return hist

### Main function that masks the image and gets two color histograms (one for the input and output image). It then returns the dot product between the two unit-normalized histograms, which is an estimate of the color similarity

#### The segmentation of the output image (out_seg_path) is an optional argument. If it is not included, the output image will be masked with the input image arm mask. While this will not match the output exactly (as the model may generate the arms slightly differently than in the input image), it will be a good approximation and removes the need to run the segmentation model a second time on the output image.

In [4]:
def get_skintone_similarity(in_pic_path, in_seg_path, out_pic_path, out_seg_path=''):
    # Load and resize images
    in_pic = misc.imread(in_pic_path)
    in_pic = misc.imresize(in_pic, (256, 192, 3))

    out_pic = misc.imread(out_pic_path)
    out_pic = misc.imresize(out_pic, (256, 192, 3))
    
    # Load segment and return arm mask
    in_mask  = get_arm_mask(in_seg_path)
    if out_seg_path:
        out_mask = get_arm_mask(out_seg_path)
    else:
        out_mask = in_mask
    
    # Get color histograms of arms
    in_hist  = get_color_histogram(in_pic, in_mask)
    out_hist = get_color_histogram(out_pic, out_mask)
    
    # Return dot product of histograms
    return np.dot(in_hist, out_hist)

### Sanity check
#### Running the check on the same input and output image should result in a score of 1.0

In [5]:
get_skintone_similarity('../inputs/example_person.jpg',
                        '../human_parsing/output/example_person.mat',
                        '../inputs/example_person.jpg',
                        '../human_parsing/output/example_person.mat')

1.0

### Compare input and output image

#### Run without parsing the output image

In [6]:
get_skintone_similarity('../inputs/example_person.jpg',
                        '../human_parsing/output/example_person.mat',
                        '../output/example_output.png')

0.52816183930138871

#### Run with parsing the output image

In [7]:
get_skintone_similarity('../inputs/example_person.jpg',
                        '../human_parsing/output/example_person.mat',
                        '../output/example_output.png',
                        '../human_parsing/output/example_output.mat')

0.59188054857848238